# NKSOPS_03_DataTransformation_02

In [ ]:
#load Enviornment
%run NKSOPS_02_ClassAndMethods_01.ipynb

In [ ]:
# Initalize
NKS_OPS_Flight_INFO = NKS_OPS(input_path = r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataFiles\FlightInfo\FullFlight\*.csv"
                                   , export_parent_dir = r"C:\Users\76350\Documents\NKS_OPS\03_Data\02_Processed_Data"
                                   , input_columns = None
                                   , export_columns = None
                                   , research_topic = 'NKS_OPS_FlIGHT_INFO'
                                   , severity = 'H'
                                   , logfile = False
                                   , sqlite_create = r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataBase\NKS_OPS_DB_sqlite.db")

NKS_OPS_DELAY_CANCEL = NKS_OPS(input_path = r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataFiles\FlightInfo\FlightDelayAndCancel\*.csv"
                                   , export_parent_dir = r"C:\Users\76350\Documents\NKS_OPS\03_Data\02_Processed_Data"
                                   , input_columns = None
                                   , export_columns = None
                                   , research_topic = 'NKS_OPS_DELAY_CANCEL'
                                   , severity = 'H'
                                   , logfile = False
                                   , sqlite_create = r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataBase\NKS_OPS_DB_sqlite.db")

In [ ]:
#1 Execute method to get impoted email files, add unique_id/ filepath, and
NKS_OPS_Flight_INFO.NKS_OPS_Data_Study()
NKS_OPS_DELAY_CANCEL.NKS_OPS_Data_Study()

#1.a Append to single dataframe and rename column
flight_info = NKS_OPS_Flight_INFO.Append_to_DF()
delay_cancel = NKS_OPS_DELAY_CANCEL.Append_to_DF()


In [ ]:
flight_info = flight_info.rename(columns= {'Deprt. Date': 'Depart_Date','Airport Code(Deprt.)':'Apt_Deprt', 'Airport Code(Arrvl.)': 'Apt_Arrvl'})
flight_info[['Depart_Day','Depart_Month','Depart_Year']] = flight_info['Depart_Date'].str.split('/',expand=True)
flight_info['Depart_Day'] = flight_info['Depart_Day'].apply('{:0>2}'.format)
flight_info['Depart_Month'] = flight_info['Depart_Month'].apply('{:0>2}'.format)
flight_info['Depart_Year'] = flight_info['Depart_Year'].apply('{:0>2}'.format)
flight_info['Flight'] = flight_info['Flight'].str.replace(',','')
flight_info['Unique_Flight'] = flight_info.Depart_Day+"_"+flight_info.Depart_Month+"_"+flight_info.Depart_Year+"_"+flight_info.Flight+"_"+flight_info.Apt_Deprt+"_"+flight_info.Apt_Arrvl+"_"+flight_info.Tail.astype(str)
flight_info['Unique_Flight_Deprt'] = flight_info.Depart_Day+"_"+flight_info.Depart_Month+"_"+flight_info.Depart_Year+"_"+flight_info.Apt_Deprt+"_"+flight_info.Tail.astype(str)
flight_info['Unique_Flight_Arrvl'] = flight_info.Depart_Day+"_"+flight_info.Depart_Month+"_"+flight_info.Depart_Year+"_"+flight_info.Apt_Arrvl+"_"+flight_info.Tail.astype(str)

delay_cancel = delay_cancel.rename(columns= {'Deprt. Date': 'Depart_Date','Airport Code(Deprt.)':'Apt_Deprt', 'Airport Code(Arrvl.)': 'Apt_Arrvl','Actual Delay':'Actual_Delay','Delay Code':'Delay_Code'})
delay_cancel[['Depart_Day','Depart_Month','Depart_Year']] = delay_cancel['Depart_Date'].str.split('/',expand=True)
delay_cancel['Depart_Day'] = delay_cancel['Depart_Day'].apply('{:0>2}'.format)
delay_cancel['Depart_Month'] = delay_cancel['Depart_Month'].apply('{:0>2}'.format)
delay_cancel['Depart_Year'] = delay_cancel['Depart_Year'].apply('{:0>2}'.format)
delay_cancel['Flight'] = delay_cancel['Flight'].str.replace(',','')
delay_cancel['Unique_Flight'] = delay_cancel.Depart_Day+"_"+delay_cancel.Depart_Month+"_"+delay_cancel.Depart_Year+"_"+delay_cancel.Flight+"_"+delay_cancel.Apt_Deprt+"_"+delay_cancel.Apt_Arrvl+"_"+delay_cancel.Tail.astype(str)


In [ ]:
merged_flights = flight_info.merge(delay_cancel[['Unique_Flight','Actual_Delay','Delay_Code']],how='left',on='Unique_Flight')
move_col = merged_flights.columns.tolist()
move_col.insert(1, move_col.pop(move_col.index('Unique_Flight')))
move_col.insert(2, move_col.pop(move_col.index('Unique_Flight_Deprt')))
move_col.insert(3, move_col.pop(move_col.index('Unique_Flight_Arrvl')))
move_col.insert(4, move_col.pop(move_col.index('Actual_Delay')))
move_col.insert(5, move_col.pop(move_col.index('Delay_Code')))
move_col.insert(15, move_col.pop(move_col.index('FILE_PATH')))
merged_flights = merged_flights.reindex(columns= move_col).drop(columns=(['Unique_ID','ID'])).rename_axis('ID').reset_index()
merged_flights['Unique_Flight'] = merged_flights['Unique_Flight'].astype(str) + "_"+merged_flights['Delay_Code'].astype(str)+ "_"+merged_flights['ID'].astype(str)
display(merged_flights.head())


In [ ]:
#export to sqlite3
NKS_OPS_Flight_INFO.NKS_OPS_DB(db_file=NKS_OPS_Flight_INFO.sqlite_create)
num_rows = len(merged_flights)
conn = NKS_OPS_Flight_INFO.create_connection(db_file = NKS_OPS_Flight_INFO.sqlite_create)
cur = conn.cursor()
for opsrow in range(num_rows):
    try:
        merged_flights.iloc[opsrow:opsrow+1].to_sql(name='NKS_FLIGHTS', con=conn, if_exists='append', chunksize=10000, index=False)
    except exc.IntegrityError:
        #Ignore duplicates
        pass

In [ ]:
conn.close()